Importing data from tinkoff

In [67]:
import tinkoff.invest as ti
from configparser import ConfigParser
import pandas as pd
import datetime, time
import psycopg2
from io import StringIO
import csv
import telebot

In [68]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
today_date = datetime.date.today()
today_date_start = datetime.datetime(today_date.year, today_date.month, today_date.day)

In [69]:
config_parser = ConfigParser()
config_parser.read('../airflow/tinkoff.cfg')
token = config_parser.get('core', 'TOKEN_TINKOFF')

In [70]:
bot = telebot.TeleBot(config_parser.get('tg', 'TOKEN_TG'))
chatid = config_parser.get('tg', 'CHAT_ID')

In [71]:
def get_figi(name):
    instruments = client.instruments.futures()
    for instrument in instruments.instruments:
        if instrument.ticker == name:
            break
    return instrument.figi

In [72]:
def calcDirection(open, close):
    if open==close:
        return '='
    if close-open>0:
        return '+'
    if close-open<0:
        return '-'
    else:
        return 'err'

In [73]:
def create_df(candles: [ti.HistoricCandle]):
    df = pd.DataFrame ([{
        'time' : c.time+datetime.timedelta(hours=3),
        'open' : c.open.units+c.open.nano/1000000000,
        'close' : c.close.units+c.close.nano/1000000000,
        'low' : c.low.units+c.low.nano/1000000000,
        'high' : c.high.units+c.high.nano/1000000000,
        'diff' : abs(c.close.units-c.open.units),
        'dir' : calcDirection(c.open.units+c.open.nano/1000000000,c.close.units+c.close.nano/1000000000)
    } for c in candles])
    return df

In [74]:
    with ti.Client(token=token) as client:
        figi = get_figi('RIM2')
        r = client.users.get_accounts()
        r = client.market_data.get_candles(
            figi=figi,
            from_=datetime.datetime.now() - datetime.timedelta(hours=4),
            to = datetime.datetime.now(),
            interval=ti.CandleInterval.CANDLE_INTERVAL_HOUR
        )
        df = create_df(r.candles)

In [75]:
df

,time,open,close,low,high,diff,dir
0,2022-05-18 13:00:00+00:00,120790.0,120740.0,120480.0,121110.0,-50,-
1,2022-05-18 14:00:00+00:00,120730.0,119680.0,119600.0,120890.0,-1050,-
2,2022-05-18 15:00:00+00:00,119680.0,121050.0,119410.0,121610.0,1370,+
3,2022-05-18 16:00:00+00:00,121060.0,118750.0,118750.0,121400.0,-2310,-
4,2022-05-18 17:00:00+00:00,118750.0,119330.0,118620.0,119460.0,580,+


In [77]:
percent1 = df.iloc[0].open/110
percent02 = df.iloc[0].open/1000*2
if (df.iloc[0].dir==df.iloc[1].dir and df.iloc[2].dir==df.iloc[3].dir and df.iloc[1].dir!=df.iloc[2].dir \
and (df.iloc[1].diff)>percent1 and ((df.iloc[1].diff)-(df.iloc[2].diff))<percent02 \
and (abs(df.iloc[1].open-df.iloc[1].high)<percent02 or abs(df.iloc[1].close-df.iloc[1].high)<percent02) \
and (abs(df.iloc[2].open-df.iloc[2].high)<percent02 or abs(df.iloc[2].close-df.iloc[2].high)<percent02) \
and (abs(df.iloc[1].open-df.iloc[1].low)<percent02 or abs(df.iloc[1].close-df.iloc[1].low)<percent02) \
and (abs(df.iloc[2].open-df.iloc[2].low)<percent02 or abs(df.iloc[2].close-df.iloc[2].low)<percent02)):
    bot.send_message(chatid,'SEE RIH')



Importing to DB

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   time    191 non-null    datetime64[ns, UTC]
 1   open    191 non-null    float64            
 2   close   191 non-null    float64            
 3   low     191 non-null    float64            
 4   high    191 non-null    float64            
 5   dir     191 non-null    object             
dtypes: datetime64[ns, UTC](1), float64(4), object(1)
memory usage: 9.1+ KB


In [14]:
login='trader'
password='income'
host='localhost'
port=5432
dbname='stocks'


In [15]:
DB_URL = f'user={login} password={password} host={host} port={port} dbname={dbname}'

In [16]:
DB_URL

'user=trader password=income host=localhost port=5432 dbname=stocks'

In [17]:
def load_df_to_db(df: pd.DataFrame, table_name:str) -> None:
    buffer= StringIO()
    df.to_csv(buffer, index=False, sep='|', na_rep='NUL', quoting=csv.QUOTE_MINIMAL, header=False, float_format='%.8f', 
              doublequote=False, escapechar='\\')
    buffer.seek(0)
    copy_query = f"""
        COPY {table_name}({','.join(df.columns)})
        FROM STDIN
        DELIMITER '|'
        NULL 'NUL'
    """
    conn = psycopg2.connect(dsn = DB_URL)
    with conn.cursor() as cursor:
        cursor.copy_expert(copy_query,buffer)
        conn.commit()
        conn.close()

In [18]:
load_df_to_db(df,'data')